Data can be found here:

https://www.kaggle.com/uciml/human-activity-recognition-with-smartphones/downloads/human-activity-recognition-with-smartphones.zip  

In [278]:
import numpy
import pandas
import seaborn
import matplotlib.pyplot as plt

import numpy as np
import pandas as pd
import seaborn as sns

from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

## 2 - Load and analyse the data

In [2]:
train = pd.read_csv("./har_data/train.csv")
test = pd.read_csv("./har_data/test.csv")

In [3]:
train.head()

,tBodyAcc-mean()-X,tBodyAcc-mean()-Y,tBodyAcc-mean()-Z,tBodyAcc-std()-X,tBodyAcc-std()-Y,tBodyAcc-std()-Z,tBodyAcc-mad()-X,tBodyAcc-mad()-Y,tBodyAcc-mad()-Z,tBodyAcc-max()-X,...,fBodyBodyGyroJerkMag-kurtosis(),"angle(tBodyAccMean,gravity)","angle(tBodyAccJerkMean),gravityMean)","angle(tBodyGyroMean,gravityMean)","angle(tBodyGyroJerkMean,gravityMean)","angle(X,gravityMean)","angle(Y,gravityMean)","angle(Z,gravityMean)",subject,Activity
0,0.288585,-0.020294,-0.132905,-0.995279,-0.983111,-0.913526,-0.995112,-0.983185,-0.923527,-0.934724,...,-0.710304,-0.112754,0.030400,-0.464761,-0.018446,-0.841247,0.179941,-0.058627,1,STANDING
1,0.278419,-0.016411,-0.123520,-0.998245,-0.975300,-0.960322,-0.998807,-0.974914,-0.957686,-0.943068,...,-0.861499,0.053477,-0.007435,-0.732626,0.703511,-0.844788,0.180289,-0.054317,1,STANDING
2,0.279653,-0.019467,-0.113462,-0.995380,-0.967187,-0.978944,-0.996520,-0.963668,-0.977469,-0.938692,...,-0.760104,-0.118559,0.177899,0.100699,0.808529,-0.848933,0.180637,-0.049118,1,STANDING
3,0.279174,-0.026201,-0.123283,-0.996091,-0.983403,-0.990675,-0.997099,-0.982750,-0.989302,-0.938692,...,-0.482845,-0.036788,-0.012892,0.640011,-0.485366,-0.848649,0.181935,-0.047663,1,STANDING
4,0.276629,-0.016570,-0.115362,-0.998139,-0.980817,-0.990482,-0.998321,-0.979672,-0.990441,-0.942469,...,-0.699205,0.123320,0.122542,0.693578,-0.615971,-0.847865,0.185151,-0.043892,1,STANDING


## FIXME: group folds on subject

In [7]:
train.drop('subject', axis =1, inplace=True)
test.drop('subject', axis =1, inplace=True)

In [33]:
for x in [train, test]:
    x['Activity'] = x.Activity.astype("category")

## 4 - Splitting the data into train and validation 

In [189]:
#Getting the split indexes

split_data = StratifiedShuffleSplit(n_splits = 1, test_size = 0.3, random_state = 42)
train_idx, val_idx = next(split_data.split(train[feature_cols], train.Activity))

#creating the dataframes

x_train = train.loc[train_idx, feature_cols]
y_train = train.loc[train_idx, 'Activity']

x_val = train.loc[val_idx, feature_cols]
y_val = train.loc[val_idx, 'Activity']

## 5 - Predictive Modelling

In [177]:
from sklearn.feature_selection import VarianceThreshold

v = VarianceThreshold(threshold=0.2)
x_train = v.fit_transform(x_train)
x_val = v.transform(x_val)
x_val.shape

(2206, 58)

In [178]:
from sklearn.svm import LinearSVC
from sklearn.feature_selection import SelectFromModel

lsvc = LinearSVC(C=0.001, penalty="l1", max_iter=10000, dual=False).fit(StandardScaler().fit_transform(x_train), y_train)
model = SelectFromModel(lsvc, prefit=True)
xx = model.transform(x_train)
xx.shape


(5146, 18)

In [279]:

splitter = StratifiedShuffleSplit(n_splits = 3, test_size = 0.3, random_state = 42)


KeyboardInterrupt: 

In [181]:
from sklearn.model_selection import cross_val_score
rf = RandomForestClassifier(n_estimators = 30)

cross_val_score(rf, x_train, y_train)

array([0.91747573, 0.93197279, 0.91836735, 0.93683188, 0.92322643])

In [93]:
x_train.shape

(5146, 58)

In [157]:
from sklearn.svm import NuSVC, SVC
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

svc = NuSVC(nu=0.10,
      kernel='linear',
      #gamma='scale',
      shrinking=True,
      tol=0.001,
      cache_size=200,
      verbose=False,
      max_iter=100000,
      decision_function_shape='ovr',
      break_ties=False,
      random_state=None)

pipe = make_pipeline(StandardScaler(), svc)

scores = cross_val_score(pipe, x_train, y_train)
print(scores)
pipe.fit(x_train, y_train)
print(np.sum(pipe.named_steps['nusvc'].n_support_))


[0.75145631 0.63848397 0.39067055 0.52380952 0.71137026]
1388


In [ ]:
# Seems like NuSVC has a quite small range for the support vectors

In [164]:
from sklearn.svm import LinearSVC
from sklearn.feature_selection import SelectFromModel

svc = SVC(C=100.0, max_iter=100000, kernel='linear')
pipe = make_pipeline(StandardScaler(), svc)
scores = cross_val_score(pipe, x_train, y_train)
print(scores)
pipe.fit(x_train, y_train)
print(pipe.named_steps)
print(np.sum(pipe.named_steps['svc'].n_support_))


[0.84757282 0.83187561 0.85519922 0.84645287 0.86297376]
{'standardscaler': StandardScaler(), 'svc': SVC(C=100.0, kernel='linear', max_iter=100000)}
1869


In [ ]:
# LinearSVC has both a larger range for the support vectors, and a lower number for better performance